In [ ]:
from utils.OBJ_helper import OBJ
import os
import numpy as np
import trimesh
from utils.pickel_io import load_from_memory
from utils.Blendshape import ZeroMeanDefMatrix
from scipy import sparse
from scipy import linalg
from scipy.sparse.linalg import splu
from utils.Geodesic_dist import compute_topological_laplacian
from utils.vis_tools import VisPointsAttributes
save_path = "../dataset/multiface/tracked_mesh"

In [ ]:
from datetime import datetime
# get current date and year
now = datetime.now()

date = now.strftime("%d") + now.strftime("%m") + now.strftime("%Y")
print(date)
time = now.strftime("%H_%M")
print("time:", time)

# Load meshes in trimesh

In [ ]:
import sys
import os
# set path to dataset
path_to_dataset = os.path.join(os.getcwd(), '../dataset/multiface/tracked_mesh/')

ID = 6795937

In [ ]:
# create a list of expressions
list_exps_name = []
map_exps2id = {}
counter = 0
for i, name in enumerate(os.listdir(path_to_dataset)):
    f = os.path.join(path_to_dataset, name)
    if os.path.isdir(f) and name.startswith('E0'):
        counter = counter + 1
        list_exps_name.append(name)


list_exps_name.sort()

for i, exp_name in enumerate(list_exps_name):
    print(f'{i}, {exp_name}')
    map_exps2id.update({exp_name: i})

In [ ]:
from utils.Dataset_handler import Filehandler

Get list of .obj file name in respective expression folder

In [ ]:
file_handler = Filehandler(path_to_dataset=path_to_dataset)
print(file_handler.get_path_to_dataset())
file_handler.iter_dir()
print("Expressions: Number of tracked mesh")
for key in file_handler.dict_objs.keys():
    print(f'{list_exps_name[key]}: {len(file_handler.dict_objs[key])}')
    print(file_handler.dict_objs[key]) 
# print(file_handler.dict_objs)

Load mesh using trimesh or original mesh loader

In [ ]:
mesh_loader = "trimesh"
# mesh_loader = "original"

In [ ]:
dict_expMeshes = {}
dict_expVerts = {}

# selecgt number of samples for a expression 
num_samples_perExp = 20

for expID, key in enumerate(file_handler.dict_objs.keys()):
    list_Meshes = []
    list_Verts = []

    # since there are many sequences for a expression, we assume that second half of tracked mesh in a sequence captured the specific expressions
    # We only use second half of sequence for a expression.
    # This is also for resonable memory usage as well. if you run over all, you will consume more than 30GB memory to store all of objects

    # half_id = int(len(file_handler.dict_objs[key])/2)
    # end_id = int(len(file_handler.dict_objs[key]))

    for i, obj in enumerate(file_handler.dict_objs[key][0:num_samples_perExp]):
        path_to_obj = os.path.join(file_handler.list_expPathFiles[expID], obj)
        # print(path_to_obj)
        if mesh_loader == "trimesh":
            _mesh = trimesh.load(path_to_obj, force='mesh')
        elif mesh_loader == "original":
            _mesh = OBJ(path_to_obj, swapyz=False)

        list_Meshes.append(_mesh)
        list_Verts.append(_mesh.vertices)
    dict_expMeshes.update({expID: list_Meshes})
    dict_expVerts.update({expID: list_Verts})

The number of vertices

In [ ]:
num_vertices = len(dict_expVerts[0][0])
len_col = num_vertices * 3
print(f"Number of vertex: {num_vertices}")
print(f"The length of column: {len_col}")

Concatenate all vertex lists

In [ ]:
_list_xs = []
num_sum_samples = 0
for key in dict_expVerts.keys():
    vertices = dict_expVerts[key]
    _num_samples = len(vertices)
    # print(_num_samples)
    num_sum_samples = num_sum_samples + _num_samples
    # shape = [F, N*3]
    # _array = np.array(vertices).reshape((_num_samples, len_col))
    # shape = [F, N, 3]
    _array = np.array(vertices)
    _list_xs.append(_array)

neutralmesh_verts = _list_xs[0]
X = _list_xs[0]
for x in _list_xs[1:]:
    X = np.concatenate((X, x), axis = 0)
    # print(X)
print(X.shape)

Obtain neutral face mesh vertex list

In [ ]:
ave_neutralmesh_vertices = np.mean(neutralmesh_verts, axis = 0)
print(ave_neutralmesh_vertices.shape)

Centralized vertex coordinate at the neutral face mesh vertex position

In [ ]:
cent_X = X - ave_neutralmesh_vertices[None, :]

Get standard deviation

In [ ]:
centX_std = np.std(cent_X)

Face mask
- Masking the vertices to take into account only vertices composing the front face
- To do it, we only perform methodology over masked region

In [ ]:
from utils.Blendshape import FaceMask
from utils.pickel_io import dump_pckl, load_from_memory

# set the name of pickel file to be loaded
if mesh_loader == "trimesh":
    pickel_fname = "FaceMask_29112023_11_23_trimesh.pkl"
elif mesh_loader == "original":
    pickel_fname = "FaceMask_30102023_09_40.pkl"

facemask = load_from_memory(path_to_memory = save_path, pickle_fname = pickel_fname)


In [ ]:
print(facemask.bit_mask.shape)

In [ ]:
masked_cent_X = cent_X * facemask.bit_mask[None, :]

In [ ]:
mesh = trimesh.load(os.path.join(save_path, "sample.obj"), force='mesh')
tris = np.asarray(mesh.faces)

In [ ]:
# dump the data matrix if you need to dump the matrix to save loading time
deformation_data = ZeroMeanDefMatrix(masked_cent_x = masked_cent_X, mean = ave_neutralmesh_vertices, std = centX_std, tris = tris)
dd_pickel_fname = 'deformation_data_matrix_and_mean'+ '_' +date+'_'+time+'_'+mesh_loader+'.pkl'
dump_pckl(data = deformation_data, save_root= save_path, pickel_fname=dd_pickel_fname)

# Sparse localized deformation components
- input: deformation matrix (zero-mean) 
$$\mathbf{X} \text{ (shape = [\#trackedMeshes, \#Vertices])}$$

- output: sparse localized deformation component (shape = [#components, #vertices])
$$\mathbf{C} \text{ (shape = [\#Components, \#Vertices])}$$
    This is from the matrix factorization inducing sparsity in matrix $C$

In [ ]:
import os
import numpy as np
import trimesh

from utils.OBJ_helper import OBJ
from utils.Blendshape import FaceMask
from utils.Blendshape import ZeroMeanDefMatrix
from utils.Geodesic_dist import compute_topological_laplacian
from utils.vis_tools import VisPointsAttributes
from utils.pickel_io import dump_pckl, load_from_memory
from utils.Geodesic_dist import GeodesicDistHeatMethod, GeodesicDistSimple, compute_support_map
from utils.converter import vector2MatNx3
from utils.common_utils import project_weight, proxy_l1l2

from scipy import sparse
from scipy import linalg
from scipy.sparse.linalg import splu
save_path = "../dataset/multiface/tracked_mesh"

# use efficient implementation of sparse Cholesky factorization.
from sksparse.cholmod import cholesky_AAt, cholesky


In [ ]:
deformation_data_pickel_fname = "deformation_data_matrix_and_mean_29112023_11_17_trimesh.pkl"
# load the deformation data matrix
deformation_data = load_from_memory(path_to_memory=save_path, pickle_fname=deformation_data_pickel_fname)

In [ ]:
# masked_cent_X: after centralized at mean, masked by bit mask
masked_cent_X = deformation_data.masked_cent_x

# MEAN: original vertex list of neutral face mesh
MEAN = deformation_data.mean

# std: standard deviation of cent_X(before masked)
std = deformation_data.std

# tris: triangle list (index tuples for triangle mesh)
tris = deformation_data.tris

Nverts = int(MEAN.shape[0]/3)
if deformation_data_pickel_fname.endswith("_trimesh.pkl"):
    mesh_loader = "trimesh"
else:
    mesh_loader = "original"

In [ ]:
print(f"shape of data matrix: {masked_cent_X.shape}")
print(f"shape of mean mesh vertex array: {MEAN.shape}")
print(f"shape of triangle list: {tris.shape}")
print(f"std of data matrix: {std}")
print(f"Number of vertices: {Nverts}")
print(f"Mesh loader: {mesh_loader}")

Visualize mean mesh

In [ ]:
# from utils.converter import vector2MatNx3
# _MEAN_MatNx3 = vector2MatNx3(MEAN, Nverts)
# VisPointsAttributes(_MEAN_MatNx3, None, cmap = 'jet')

# Support Region Computation

Obtain triangle list
- Since the tracked meshes are topologically equivalent, we can get triangle list in advance from a sample.obj

- option1:load obj file using original loader

In [ ]:
# # Get neutral face vertices and neutral face triangle list
# target_obj = OBJ(filename = os.path.join(save_path, "sample.obj"))

# list_vertices = target_obj.vertices
# list_triangles = target_obj.faces

# verts = np.asarray(list_vertices)

# CENTER = 3567

# tris = []
# for triangle in list_triangles:
#     # 0: triangle index list
#     # 1: normals
#     # 2: texture coordinate
#     # 3: material configuration
#     tris.append(triangle[0])
# tris = np.asarray(tris)

- option2:load obj file using trimesh loader

In [ ]:
mesh = trimesh.load(os.path.join(save_path, "sample.obj"), force='mesh')
list_vertices = mesh.vertices
list_triangles = mesh.faces

verts = np.asarray(list_vertices)
tris = np.asarray(list_triangles)

CENTER = 2658

In [ ]:
# verts: (N, 3) array (float)
# tris: (m, 3) array (int): indices into the verts array
print(tris.shape)
print(verts.shape)

Triangle list conversion
- index should be start from 0 to #num_vertex

In [ ]:
if tris.min() > 0:
    for triangle in tris:
        for i in range(3):
            # print(triangle)
            triangle[i] = triangle[i] - int(1)

Obtain distance function

In [ ]:
# heat method
gdd = GeodesicDistHeatMethod(verts, tris)
phi_heat = gdd(CENTER) #the vertex on top of a nose
# visualize support map
# gdd.visualize_distance_func()

# simple method
# simple_gdd = GeodesicDistSimple(verts=verts, tris=tris)
# phi_simple = simple_gdd(CENTER)
# simple_gdd.visualize_distance_func()


Generate support map (coefficient assignment)

In [ ]:
# nornalized distance function
Nphi_heat = phi_heat / max(phi_heat)
min_dist = 0.05
max_dist = 0.35
support_map = compute_support_map(Nphi_heat, min_dist, max_dist)

visualize support map (source #2658)

In [ ]:
# visualize support map
VisPointsAttributes(verts, support_map, cmap = 'coolwarm')

# Pre computation
- normalized masked/centralized vertex position into [-0.5, 0.5]

In [ ]:
preScaleFactor = 1/std
# Nmasked_cent_X = masked_cent_X*preScaleFactor
N_cent_X = cent_X * preScaleFactor
# R = Nmasked_cent_X.copy()
R = N_cent_X.copy()
print(R.shape)

In [ ]:
# number of components
Ncompos = 300

# minimum/maximum geodesic distance for support region 
srMinDist = 0.1
srMaxDist = 0.5

# number of iterations to run
num_iters_max = 100

# sparsity parameter (coeffient lambda for weight of L1 regularization term)
sparse_lambda = 2.

# pernalty parameter for ADMM (for multiplier)
# Choice of ρ can greatly influence practical convergence of ADMM
# TOO large: not enough emphasis on minimizing a f+z
# TOO small: not enought emphasis on feasibility (Ax+Bz = c) 
rho = 10.0

# number of iteration of ADMM
num_admm_iterations = 10

# geodesic distance computation on the mean verts
gdd = GeodesicDistHeatMethod(MEAN, tris)

# Initialization
- Use deflation algorithm
- for `k` in `num_components`
    - Initialize $\mathbf{W,C} = \mathbf{0}$ and $\mathbf{R} = \mathbf{X}$
    - Find the vertex j with the highest residual in matrix $\mathbf{R}$
    $$ j = \text{argmax}_{j} \mathbf{X} - \mathbf{WC}$$
    - Find the component $C_k$ and corresponding weights $W_{:,k}$ at each step that explain maximal variance in the data via SVD/PCA
    - Subtract each of them from the deformataion matrix $\mathbf{X}$ to compute residual $\mathbf{R}$


In [ ]:
C = []
W = []

for k in range(Ncompos):
    # find the vertex explaining the most variance across the residual matrix R
    # take a norm of residual at each vertex
    magnitude = (R**2).sum(axis = 2) #shape [FxN]

    # vertex id with the most variance (residual)
    idx = np.argmax(magnitude.sum(axis = 0))

    # Find linear component explaining the motion of this vertex
    # R: shape = [F, 3]
    _U, s, Vh = linalg.svd(R[:, idx, :].reshape(R.shape[0], -1).T, full_matrices=False)
    
    # reconstruct column of matrix W at K-th column using most variant direction
    w_k = s[0] * Vh[0, :]

    # invert weight according to their projection onto the constraint set
    # This prevent problems from having negative weights
    wk_proj = project_weight(w_k)
    wk_proj_negative = project_weight(-1*w_k)

    # W_k will be replaced by the larger variance direction (+ or -)
    if(linalg.norm(wk_proj) > linalg.norm(wk_proj_negative)):
        w_k = wk_proj
    else:
        w_k = wk_proj_negative

    # flipped support region
    phi = gdd(idx)
    phi/=max(phi)
    flippedSR = 1 - compute_support_map(phi, srMinDist, srMaxDist)

    # Solve normal equation to get C_k
    # R: shape = [F, N, 3]
    # W_K: shape = [F, 1]
    # c_k: shape = [N, 3]
    # flippedSR: shape = [N, ]
    # W_k*C_k = flippedSR*R
    # C_k = (W_k^T*W_k)^{-1} W_k^T*flippedSR*R

    c_k = (np.tensordot(w_k, R, (0, 0)) * flippedSR[:, None])/ np.inner(w_k, w_k)

    C.append(c_k)
    W.append(w_k)

    # update residual
    R = R - np.outer(w_k, c_k).reshape(R.shape)

C = np.array(C) #shape = [K, N, 3]
W = np.array(W).T #shape = [F, K]


In [ ]:
print(C.shape)
print(W.shape)

# Optimization for matrix W (coefficient matrix)
- The optimization problem w.r.t matrix W is separable due to the additional constraint
- The constraints act on the weight vector $\mathbf{W_{:, k}}$ of each component separately.
- Use the block-coordinate descent algorithm, which optimize each column successively.
- Then project the updated each column of W by projecting them onto the desired W space
$$W'_{:, k} = \text{argmin}_{\mathbf{W_{:, k}\in \mathcal{V}}} ||\mathbf{X} - \mathbf{WC}||_{F}^2 = \frac{(\mathbf{R} + W_{:, k} C_k)\cdot C_k}{C_k^TC_k}$$
$$W' = \frac{W'}{\text{max}(W')}$$
$$\text{where } \mathbf{R} = \mathbf{X} - \mathbf{WC}$$


# Optimization for matrix C (deformation matrix)
- Where we fixed matrix $\mathbf{W}$, we can optimize C using convex optimization
- Use ADMM (Alternating direction method of multipliers)
    - This can optimize matrix C with good robostness of method of multipliers (faster than dual decomposition)
    - This supports decomposition (method of multipliers does not support decomposition due to the quadoratic penalty)
- Lasso problem $\mathbf{Z}$
$$\text{argmin}_{\mathbf{C}, \mathbf{Z}} ||\mathbf{X} - \mathbf{W} \cdot \mathbf{C}||_{F}^2 + \Omega(\mathbf{Z})$$
$$\text{s.t. } \mathbf{C} - \mathbf{Z} = 0$$
- Augumented Lagragian (Lagragian of ADMM)
$$\text{argmin}_{\mathbf{C}, \mathbf{Z}} ||\mathbf{X} - \mathbf{W} \cdot \mathbf{C}||_{F}^2 + \Omega(\mathbf{Z}) + \mathbf{Y}^T(\mathbf{C}-\mathbf{Z})+ (\frac{\rho}{2})||\mathbf{C}-\mathbf{Z}||_2^2$$
$$= \text{argmin}_{\mathbf{C}, \mathbf{Z}} ||\mathbf{X} - \mathbf{W} \cdot \mathbf{C}||_{F}^2 + \Omega(\mathbf{Z}) + (\frac{\rho}{2})||\mathbf{C}-\mathbf{Z} + \mathbf{U}||_2^2$$
$$\text{where } \mathbf{U} = (\frac{1}{\rho})\mathbf{Y}$$

- The ADMM algorithm initializes $\mathbf{U}\in \real^{K \times 3N}$ to zero and then iterates the following steps.
- Dual ascent
$$C^* = \text{argmin}_C ||X-WC||_{F}^2 + \frac{\rho}{2}||\mathbf{C}-\mathbf{Z}+\mathbf{U}||_{F}^2 = (W^TW + \rho I)^{-1} (W^TX+\rho(Z-U))$$
$$Z^* = \text{argmin}_Z (\Omega(\mathbf{Z}) + \frac{\rho}{2}||\mathbf{C^*}-\mathbf{Z}+\mathbf{U}||_{F}^2) = proxy_{\rho}(0, (1-\frac{\Lambda_{i,k}}{\rho ||\mathbf{C}^* + \mathbf{U}||_2^2}))_{+}[\mathbf{C}^* + \mathbf{U}]$$
- Dual update
$$\mathbf{U}^* = \mathbf{U} + \mathbf{C}^* - \mathbf{Z}^*$$


In [ ]:
X.shape

In [ ]:
# Nmasked_cent_X.shape
N_cent_X.shape

In [ ]:
# original_sparsity = np.sum( * np.sqrt((C**2).sum(axis = 2)))
# original_error = (Nmasked_cent_X**2).sum()
original_error = (N_cent_X**2).sum()
print(original_error)

In [ ]:
# global optimization
# F, N, _ = Nmasked_cent_X.shape
F, N, _ = N_cent_X.shape

Lambda = np.empty((Ncompos, N)) # each row representing the scaler of l1 penalty depending on the locality
U = np.zeros_like(C)
print(U.shape)

for i in range(num_iters_max):
    # Update weights
    # fix weight matrix, optimize C (each row respectively: c_k)
    Rflat = R.reshape(F, N*3) #flattened residual, shape = [F, N*3]
    for k in range(C.shape[0]): # for c_k (kth row)
        c_k = C[k].ravel() #flatten into [1, N*3]
        ck_norm = np.inner(c_k, c_k)
        if ck_norm <= 1e-8: # if the component does not represent any deformation component
            W[:, k] = 0
            continue # to prevent dividing by 0
        
        #block coordinate descent update
        # get updated W[:,k]'
        Rflat += np.outer(W[:, k], c_k) 
        opt = np.dot(Rflat, c_k) / ck_norm 

        #project W onto the desired space from constraints
        W[:, k] = project_weight(opt)
        Rflat -= np.outer(W[:, k], c_k)

    # precomputing lambda for each component k (Regularization term)
    # spatially varying regularization strength (to encode locality)
    for k in range(Ncompos):
        ck = C[k] #not flatten
        # find vertex with the biggest displacement in component and computer support map around it
        # take displacement vector norm at each vertex and find index with maximum of norm
        idx = (ck**2).sum(axis = 1).argmax()
        phi = gdd(idx)
        phi/=max(phi)
        support_map = compute_support_map(phi, srMinDist, srMaxDist)

        # update L1 regularization strength according to this support map
        Lambda[k] = sparse_lambda * support_map
    
    # TODO
    # Inf or NaN check in W and C

    # update components
    Z = C.copy() # this is dual variable

    # optimize matrix C fixing W
    # prefactor linear solve in ADMM
    G = np.dot(W.T, W)
    # G[np.isfinite(G) == False] = 0
    # c = np.dot(W.T, Nmasked_cent_X.reshape(Nmasked_cent_X.shape[0], -1)) #Nmasked_cent_X.reshaped into [F, N*3]  
    c = np.dot(W.T, N_cent_X.reshape(N_cent_X.shape[0], -1)) #Nmasked_cent_X.reshaped into [F, N*3]  
    # compute inverse part
    # scipy
    solve_prefactored = linalg.cho_factor(G + rho * np.eye(G.shape[0]))

    # sksparse.cholmod
    # sparse_csc_c = sparse.csc_matrix(G + rho * np.eye(G.shape[0]))
    # solve_prefactored = cholesky(sparse_csc_c)

    # ADMM iterations
    # TODO
    #    - check cho_factor and cho_solve from scipy
    #    - create function for proxy of update of l1/l2 reguralization term
    # old_U = U.reshape(U.shape[0], -1)
    for admm_it in range(num_admm_iterations):
        # temp_U = U.reshape(U.shape[0], -1)
        # for i in range(temp_U.shape[0]):
        #     for j in range(temp_U.shape[1]):
        #         if not np.isfinite(temp_U[i][j]):
        #             if old_U[i][j] != 0.0:
        #                 print(f"{i}, {j}: {old_U[i][j]}")
        rhs = c + rho * (Z.reshape(c.shape) - U.reshape(c.shape))
        # rhs[np.isfinite(rhs)==False] = 0
        C = linalg.cho_solve(solve_prefactored, rhs).reshape(C.shape)
        # sparse_csc_rhs = sparse.csc_matrix(c + rho * (Z.reshape(c.shape) - U.reshape(c.shape)))
        # sparse_csc_lhs = solve_prefactored(sparse_csc_rhs)
        # C = sparse_csc_lhs.toarray().reshape(C.shape)
        Z = proxy_l1l2(Lambda, C+U, 1.0/rho)
        # old_U= U.reshape(U.shape[0], -1)
        U = U + C - Z

    # set updated components to dual Z
    C = Z

    # evaluate objective function
    # R = Nmasked_cent_X - np.tensordot(W, C, (1, 0)) # residual
    R = N_cent_X - np.tensordot(W, C, (1, 0)) # residual
    if (i == 0):
        initial_sparsity = np.sum(Lambda * np.sqrt((C**2).sum(axis = 2))) # L1 reguralization term 
        initial_reconst_error = ((X.reshape(X.shape[0], -1) - np.dot(W, C.reshape(C.shape[0], -1)))**2).sum()

    sparsity = np.sum(Lambda * np.sqrt((C**2).sum(axis = 2))) # L1 reguralization term 
    reconst_error = ((X.reshape(X.shape[0], -1) - np.dot(W, C.reshape(C.shape[0], -1)))**2).sum()
    print(f"Reconstruction error: {(reconst_error/initial_reconst_error)}")
    print(f"Sparsity: {sparsity/initial_sparsity}")
    e = ((reconst_error/initial_reconst_error)) + sparsity/initial_sparsity

    # convergence check
    print("iteration %03d, E=%f" % (i, e))


# undo scaling
C /= preScaleFactor    

In [ ]:
# np.isfinite(C).all()
# np.isfinite(W).all()
# # solve_prefactored[0]
# np.isfinite(solve_prefactored[0]).all()
# U[np.isfinite(U)==False]
# np.isfinite(G).all()

In [ ]:
C = C * facemask.bit_mask[None, :]

In [ ]:
print(W.shape)
print(C.reshape(C.shape[0], -1).shape)

In [ ]:
print(np.isfinite(W).all())
print(np.isfinite(C).all())
print(np.isfinite(solve_prefactored[0]).all())

In [ ]:
for i in range(C.shape[0]):
    print(C[i].ravel())
C[0].max()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(C.reshape(C.shape[0], -1).T)

In [ ]:
# Soarsity check
# close to 1: Sparse, close to 0: Dense
sparsity_level = np.mean(C==0)
print(sparsity_level)

# export blenshape components
```
@dataclass
class datastruct_blendshape:
    ID: int
    List_exps: list
    MEAN: np.ndarray
    PCs: np.ndarray
    Stds: np.ndarray
```


In [ ]:
print(save_path)

In [ ]:
from utils.Blendshape import datastruct_blendshape
# load expression list
example_data = load_from_memory(path_to_memory=save_path, pickle_fname='blendshape_SparsePCA_07112023_16_55.pkl')

In [ ]:
max_coefficients = np.max(W, axis = 0)
min_coefficients = np.min(W, axis = 0)
print(max_coefficients.shape)
print(max_coefficients)
print(min_coefficients.shape)
print(min_coefficients)


In [ ]:
W[:, 0].max()

In [ ]:
_export_ID = date+time
_export_List_exps = example_data.List_exps
_export_MEAN = MEAN.reshape(-1)
_export_PCs = C.reshape(Ncompos,-1)
_export_Stds = max_coefficients


In [ ]:
print(_export_ID)
print(_export_List_exps)
print(_export_MEAN.shape)
print(_export_PCs.shape)
print(_export_Stds.shape)

In [ ]:
SLDC_blendshape = datastruct_blendshape(ID = _export_ID, List_exps=_export_List_exps, MEAN=_export_MEAN, PCs=_export_PCs, Stds=_export_Stds)

In [ ]:
pickel_fname_SLDC = 'SLDC_blendshape_'+date+'_'+time+'.pkl'
dump_pckl(data=SLDC_blendshape, save_root=save_path, pickel_fname=pickel_fname_SLDC)

In [ ]:
from utils.pickel_io import load_from_memory

In [ ]:
date + time

In [ ]:
test_sample = load_from_memory(path_to_memory='../dataset/multiface/tracked_mesh', pickle_fname=pickel_fname_SLDC)

In [ ]:
test_sample.PCs.shape